### Drive Mount

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
cd /content/gdrive/MyDrive/GradCAM

In [ ]:
!pip install ttach

### Dependencies

In [ ]:
import argparse
import cv2
import numpy as np
import glob
import torch
from torch import nn
from torchvision import models
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    deprocess_image, \
    preprocess_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam import GradCAM, \
    HiResCAM, \
    ScoreCAM, \
    GradCAMPlusPlus, \
    AblationCAM, \
    XGradCAM, \
    EigenCAM, \
    EigenGradCAM, \
    LayerCAM, \
    FullGrad, \
    GradCAMElementWise

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

### Load Model

In [ ]:
# VGG-16
model = models.vgg16()
model.classifier[6] = nn.Linear(4096, 4)
model.load_state_dict(torch.load('models/VGG16.pt'))
target_layers = [model.features[-1]]


#ResNet50
# model = models.resnet50()
# model.fc = nn.Linear(2048, 4)
# model.load_state_dict(torch.load('models/ResNet50.pt'))
# target_layers = [model.layer4]


model = model.to(device)
model.eval()

### Load Images

In [ ]:
images = [cv2.imread(file) for file in glob.glob('images/brain_mri/Testing/pituitary_tumor/*.jpg')]

### Create Heatmaps

In [ ]:
targets = None

i = 0

for img in images:

  # rgb_img = cv2.imread(f'images/tests/{i}.jpg', 1)[:, :, ::-1]
  rgb_img = img[:, :, ::-1]
  rgb_img = np.float32(rgb_img) / 255
  input_tensor = preprocess_image(rgb_img,
                                  mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

  input_tensor = input_tensor.to(device)
  print(i)

  cam_algorithm = GradCAM
  with cam_algorithm(model=model,
                    target_layers=target_layers,
                    use_cuda=True) as cam:

      cam.batch_size = 32
      grayscale_cam = cam(input_tensor=input_tensor,
                          targets=targets,
                          aug_smooth=False,
                          eigen_smooth=False)

      # Here grayscale_cam has only one image in the batch
      grayscale_cam = grayscale_cam[0, :]

      cam_image = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

      # cam_image is RGB encoded whereas "cv2.imwrite" requires BGR encoding.
      cam_image = cv2.cvtColor(cam_image, cv2.COLOR_RGB2BGR)

  gb_model = GuidedBackpropReLUModel(model=model, use_cuda=True)
  gb = gb_model(input_tensor, target_category=None)

  cam_mask = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])
  cam_gb = deprocess_image(cam_mask * gb)
  gb = deprocess_image(gb)

  # write heatmap to output folder
  cv2.imwrite(f'heatmaps/ResNet-50/pituitary/{i}_original.jpg', img)
  cv2.imwrite(f'heatmaps/ResNet-50/pituitary/{i}_Grad-CAM.jpg', cam_image)
  i += 1